Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [1]:
import pandas as pd

# You may need to change the path
train = pd.read_csv('./data/train_whiskey.csv')
test = pd.read_csv('./data/test_whiskey.csv')
print(train.shape, test.shape)

(4087, 3) (1022, 2)


In [2]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


In [3]:
# Distribution of ratingCategory: 0 (Excellent), 1 (Good), 2 (Poor)
train.ratingCategory.value_counts(normalize=True)

1    0.704918
0    0.279178
2    0.015904
Name: ratingCategory, dtype: float64

In [4]:
# Read a few reviews from the "Excellent" category
pd.set_option('display.max_colwidth', 0)
train[train.ratingCategory == 0].sample(3)

,id,description,ratingCategory
2543,3966,"\nNot a rum cask-finished rye, but a blend of Kentucky straight rye with Canadian rye and added blackstrap rum. The experience is sweet from start to finish: Sugar Babies and their parent Sugar Daddy on the nose, along with molasses, raisins, kettle corn, maple donuts, toasted marshmallow, and a little grassiness on the nose. The palate is sugary-sweet, with molasses, pralines, coffee ice cream, Raisinets, and some bitter oak, and it finishes short. Keep an eye on your blood sugar when drinking this.",0
3374,4290,"\nCitrus oil, freshly cut wood, tea leaves, baking spice, and maple candy on the nose; with water added, more herbal and floral notes emerge, including mint, star anise, and fresh chamomile. Medium-bodied, the palate has orange peel, licorice, star anise, brown sugar, and plenty of fresh oak; here, adding water doesn't benefit the balance. It finishes quickly, with black pepper, oak, and more star anise.",0
1953,3858,"\nWhat a surprise! Really thick and malty for a blend (there’s a high malt content here for sure), and also nicely mature for 8 years old. Great balance too, with the island aggressiveness tamed by a rich, malty elegance. Delicious honey, toasted oak, firm peat, dark chocolate, and glazed citrus meld nicely with the malt, leading to a honeyed, lightly-peppered finish. Very enjoyable.",0


In [5]:
# Read a few reviews from the "Poor" category
train[train.ratingCategory == 2].sample(3)

,id,description,ratingCategory
1525,5071,"\nThe nose on this whiskey is like smelling a wet empty barrel; it's all oak, char, and cinnamon spice. The entry is thin, watery, and understated with light caramel and cinnamon. In the mid-palate, the oak tannins completely take over. It's not just oak spice, but the puckeringly dry, sour, and bitter tannins. This all leads to a bitter, bone dry finish. Good whiskey is a conversation between spirit and wood, but here the wood is doing all the talking.",2
654,5043,"\nA blend of bourbon and rye whiskeys, 2 years old, no information on the proportions. Very hot, piercing nose: wet corn, tobacco juice, peppermint oil, and pain. Man, that’s hot. Bitter and hot on the tongue, with an inappropriate sweetness. Finish backs off on the heat, but is full of wet oak. Can’t help thinking this was simply a bad idea.",2
2032,5098,"\nI love Rogue beers, as well as Rogue’s attitude and philosophy. But this whiskey just tastes too immature—harsh, sweet, and one-dimensional. Maybe with some additional aging it might come around, mellow out, and develop, but it has a long way to go.",2


### Split the Training Set into Train/Validation

In [48]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train['description'], 
                                                    train['ratingCategory'], 
                                                    test_size=0.2, 
                                                    stratify=train['ratingCategory'],
                                                    random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(3269,) (818,) (3269,) (818,)


In [51]:
type(X_test)

pandas.core.series.Series

### Define Pipeline Components

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
clf = RandomForestClassifier()

pipe = Pipeline([('vect', vect), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [10]:
parameters = {
    'vect__max_df': (0.75, 1.0),
    'clf__max_depth': (5,10,15,20)
}

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    8.2s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

In [12]:
grid_search.best_score_

0.7060258229484243

In [20]:
grid_search.best_params_

{'clf__max_depth': 20, 'vect__max_df': 0.75}

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [13]:
# Predictions on test sample
preds = grid_search.predict(test['description'])

In [14]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory': preds})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [15]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [18]:
subNumber = 2

In [19]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 70% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [23]:
from sklearn.decomposition import TruncatedSVD

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))

svd = TruncatedSVD(algorithm='randomized',
                   n_iter=10)

clf = RandomForestClassifier()


pipe = Pipeline([
    ('vect', vect),      # TF-IDF Vectorizer
    ('lsi', svd),        # Truncated SVD Dimensionality Reduction
    ('clf', clf)         # RandomForest Classifier
])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [24]:
parameters = {
    'vect__min_df': (2, 5),
    'vect__max_df': (0.25, 0.5),
    'lsi__n_components': [100, 250],
    'clf__max_depth': (10, 20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   29.5s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:  1.1min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

In [29]:
grid_search.best_params_

{'clf__max_depth': 20,
 'lsi__n_components': 100,
 'vect__max_df': 0.25,
 'vect__min_df': 5}

In [31]:
grid_search.best_score_

0.7219377982588008

### Make a Submission File

In [25]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [26]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [27]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [28]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [32]:
# Apply to your Dataset
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from scipy.stats import randint

param_dist = {
    'max_depth' : randint(3,10),
    'min_samples_leaf': randint(2,15)
}

In [33]:
# Continue Word Embedding Work Here

import spacy
nlp = spacy.load("en_core_web_md")

In [34]:
doc = nlp("Whiskey Reviews!")

In [35]:
nlp_vector = doc.vector
print(len(nlp_vector))

300


In [37]:
nlp_vector

array([-0.30473334,  0.27565834,  0.06120667, -0.35107002,  0.11617333,
       -0.08581   , -0.16477333, -0.21034999,  0.16937001,  1.1656666 ,
       -0.34481668,  0.05642   , -0.23156832, -0.25668332, -0.09202301,
       -0.306323  , -0.02713033,  0.8628113 , -0.12996334, -0.25717735,
        0.17464101, -0.21838002,  0.14907134, -0.13064499, -0.28838366,
        0.12143935,  0.05977   ,  0.13127333,  0.35435   , -0.05882667,
        0.33437   ,  0.235557  ,  0.02755927,  0.16846967, -0.23547667,
        0.22657168,  0.14862733,  0.08206033, -0.24514066, -0.14422792,
       -0.26527   ,  0.13420063,  0.15879034, -0.17761667,  0.14228   ,
        0.46021333, -0.11348467,  0.01649866, -0.00559667, -0.09464199,
       -0.06521434,  0.05244   ,  0.09628737,  0.27949002, -0.05771834,
       -0.24813832,  0.25556666, -0.26526335, -0.04312833, -0.18449001,
        0.03358   , -0.11641733,  0.22487599,  0.12877999, -0.05396999,
       -0.02563767, -0.0063311 ,  0.02472666, -0.22070956,  0.06

In [38]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [62]:
TEST = get_word_vectors(test['description'])

In [40]:
X = get_word_vectors(X_train)
len(X) == len(X_train)

True

In [52]:
XX = get_word_vectors(X_test)
len(XX) == len(X_test)

True

In [53]:
gbd = GradientBoostingClassifier()

In [54]:
gbd.fit(X, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

### Make a Submission File

In [55]:
gbd.score(XX, y_test)

0.7383863080684596

In [63]:
# Predictions on test sample
pred = gbd.predict(TEST)

In [64]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [65]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [66]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv(f'./submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 70% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?